In [22]:
from pymatgen.io.espresso.outputs import PWxml
from pymatgen.io.espresso.inputs import PWin
#from pymatgen.io.espresso.projwfc import Projwfc
from pymatgen.io.espresso.utils import parse_pwvals
from pymatgen.io.vasp.outputs import Vasprun
from pymatgen.io.vasp.inputs import Kpoints, Poscar, Potcar
from pymatgen.electronic_structure.core import Spin
from pymatgen.electronic_structure.plotter import BSPlotter
import xml.etree.ElementTree as ET
from monty.io import zopen
import xmltodict
import numpy as np
from pymatgen.io.espresso.inputs.pwin import CellParametersCard, KPointsCard, AtomicSpeciesCard, AtomicPositionsCard, AdditionalKPointsCard

from pymatgen.core.structure import Structure


In [2]:
card = CellParametersCard("angstrom", [0, 0, 0], [0, 0, 0], [0, 0, 0])
print(card)
card = KPointsCard("automatic", [10,10,10], [1, 1, 1], [], [],[])
print(card)
card = AtomicSpeciesCard(None, ["H", "He"], [1.0, 4.0], ["H.pbe-rrkjus.UPF", "He.pbe-rrkjus.UPF"])
print(card)
card = AtomicPositionsCard("angstrom", ["He", "H"], [[0, 0, 0], [0.5, 0.5, 0.5]], None)
print(card)
card = AdditionalKPointsCard("crystal", [[0, 0, 0], [0.5, 0.5, 0.5]], [1, 1], ["X", None])
print(card)

CELL_PARAMETERS {angstrom}
   0.0000000000  0.0000000000  0.0000000000
   0.0000000000  0.0000000000  0.0000000000
   0.0000000000  0.0000000000  0.0000000000
K_POINTS {automatic}
   10  10  10   1   1   1
ATOMIC_SPECIES
    H   1.000000 H.pbe-rrkjus.UPF
   He   4.000000 He.pbe-rrkjus.UPF
ATOMIC_POSITIONS {angstrom}
   He  0.0000000000  0.0000000000  0.0000000000 1 1 1
    H  0.5000000000  0.5000000000  0.5000000000 1 1 1
ADDITIONAL_K_POINTS {crystal}
2
   0.0000000000  0.0000000000  0.0000000000    1 ! X
   0.5000000000  0.5000000000  0.5000000000    1


In [ ]:
pwin = PWin.from_file('pymatgen/io/espresso/tests/data/BAS/scf.in')
required_card_names = [c.name for c in PWinCards if c.value.required]
[pwin.cards[card] is not None for card in required_card_names]

[True, True, True]

In [ ]:
from pymatgen.io.vasp.outputs import BSVasprun
vr = BSVasprun('pymatgen/io/espresso/tests/data/Si/vasprun_bands.xml')
bs = vr.get_band_structure(line_mode=True, kpoints_filename='pymatgen/io/espresso/tests/data/Si/KPOINTS_bands')
bs.branches

[{'start_index': 0, 'end_index': 15, 'name': '\\Gamma-X'},
 {'start_index': 16, 'end_index': 31, 'name': 'X-W'},
 {'start_index': 32, 'end_index': 47, 'name': 'W-K'},
 {'start_index': 48, 'end_index': 63, 'name': 'K-\\Gamma'},
 {'start_index': 64, 'end_index': 79, 'name': '\\Gamma-L'},
 {'start_index': 80, 'end_index': 95, 'name': 'L-U'},
 {'start_index': 96, 'end_index': 111, 'name': 'U-W'},
 {'start_index': 112, 'end_index': 127, 'name': 'W-L'},
 {'start_index': 128, 'end_index': 143, 'name': 'L-K'},
 {'start_index': 144, 'end_index': 159, 'name': 'U-X'}]

In [ ]:
pwin.validate()

In [9]:
pwin = PWin.from_file('tests/data/BAS/scf.in')
pwin = PWin.from_file('tests/data/Sr3PbO/bands.in')
#pwin.structure = structure
#del pwin.k_points
#print(pwin)

In [48]:
mat = "Al"
try:
    pwin = PWin.from_file(f"tests/data/{mat}/bands.in")
except FileNotFoundError:
    pwin = PWin.from_file(f"tests/data/{mat}/scf.in")
print(pwin.system)
print(pwin.cell_parameters)
pwin.structure.to(fmt='poscar', filename=f'tests/data/{mat}/POSCAR')
pwin.structure

&SYSTEM
  ibrav = 2
  celldm = 7.5
  nat = 1
  ntyp = 1
  ecutwfc = 15.0
  nbnd = 8
/
None


/Users/ashour/code/pymatgen-io-espresso/pymatgen/io/espresso/utils.py:55: UserWarning: ibrav != 0 has not been thoroughly tested. Please be careful.
  warnings.warn("ibrav != 0 has not been thoroughly tested. Please be careful.")


Structure Summary
Lattice
    abc : 2.8063859570917136 2.8063859570917136 2.8063859570917136
 angles : 60.00000000000001 60.00000000000001 60.00000000000001
 volume : 15.628856288079671
      A : np.float64(-1.98441454088625) np.float64(0.0) np.float64(1.98441454088625)
      B : np.float64(0.0) np.float64(1.98441454088625) np.float64(1.98441454088625)
      C : np.float64(-1.98441454088625) np.float64(1.98441454088625) np.float64(0.0)
    pbc : True True True
PeriodicSite: Al (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]

 # Debugging

In [ ]:
pwin_BaAs = PWin.from_file('pymatgen/io/espresso/tests/data/BAS/scf.in')
pwin_BaAs.to_file('old.in', overwrite=True)
new_struct = PWin.from_file('pymatgen/io/espresso/tests/data/Sr3PbO/bands.in').structure
new_struct = pwin_BaAs.structure
pwin_BaAs.structure = new_struct
pwin_BaAs.to_file('new.in', overwrite=True)
#struct = pwin_BaAs.get_structure()

In [ ]:
new_struct.sites[0].species_string

In [ ]:
symbols = {s["symbol"] for s in pwin.atomic_species["data"]}
species = pwin.structure.species
sss = {s.symbol for s in set(species)}
if symbols == sss:
    print('yay')

In [ ]:
pwin_BaAs.system

In [ ]:
type(new_struct.species[0])

In [ ]:
from pymatgen.core.periodic_table import Element

In [ ]:
syms = list({site.species_string for site in new_struct})
syms

In [ ]:
poscar = Poscar(new_struct)

In [ ]:
poscar.site_symbols

In [ ]:
pwin_BaAs.set_structure(new_struct)

In [ ]:
pwin_BaAs.get_structure()

In [ ]:
pwxml_BAs = PWxml('pymatgen/io/espresso/tests/data/BAs/scf.xml')

In [ ]:
# Actual input file
alat_input = 1.889725989
# Copied from XML
alat_xml = 6.439670622385e0
a1 = np.array([0.000000000000000e0,4.553534765696103e0,4.553534765696103e0]) # units of BOHR
a2 = np.array([4.553534765696103e0,0.000000000000000e0,4.553534765696103e0]) # units of BOHR
a3 = np.array([4.553534765696103e0,4.553534765696103e0,0.000000000000000e0]) # units of BOHR
b1 = np.array([-2.075009949672709e-1,2.075009949672709e-1,2.075009949672709e-1]) # units of 2pi/alat_input
b2 = np.array([2.075009949672709e-1,-2.075009949672709e-1,2.075009949672709e-1]) # units of 2pi/alat_input
b3 = np.array([2.075009949672709e-1,2.075009949672709e-1,-2.075009949672709e-1]) # units of 2pi/alat_input

# Compute some stuff
V = np.dot(a1, np.cross(a2, a3)) # units of bohr^3
b1_computed = 2*np.pi*np.cross(a2, a3)/(V) # units of 2pi/bohr
print(f"2pi(a2xa3/V):        {b1_computed} (units of 2pi/bohr)\nb1*(2pi/alat_input): {b1*2*np.pi/alat_input} (units of 2pi/bohr)") 


In [ ]:
pwxml_BAs = PWxml('pymatgen/io/espresso/tests/data/BAs/scf.xml')

In [ ]:
alat_xml_input = 6.439670622385e0
alat_xml_output = 1.889725989000e0
bohr_to_ang = 0.5291772105638411

pwxml_BAs = PWxml("pymatgen/io/espresso/tests/data/BAs/bands.xml")
alat = pwxml_BAs.alat
np.around(pwxml_BAs.kpoints_cart, 6)# / ((2 * np.pi / alat) * (1 / bohr_to_ang)), 6)
# np.around(pwxml_BAs.kpoints_frac, 3)

In [ ]:
np.around(pwxml_BAs.kpoints_frac, 5)

In [ ]:
alat_xml_input = 6.439670622385e0
alat_xml_output = 1.889725989000e0
bohr_to_ang = 0.5291772105638411

k_input = np.array([1.625143450279226e-21,1.193947388943242e-3,-1.625143450279226e-21])
k_output = np.array([5.538045512864166e-21,4.068646973162236e-3,-5.538045512864166e-21])
ratio = alat_xml_input/alat_xml_output
k_input *= ratio**2

k_input *=  (2 * np.pi / alat_xml_input) * (1 / bohr_to_ang) # units of 1/ang
k_output *= (2 * np.pi / alat_xml_output) * (1 / bohr_to_ang) # units of 1/ang

k_output = np.around(k_output, 6)
k_input = np.around(k_input, 6)

print(f"Input:  {k_input}\nOutput: {k_output}")
print(k_output/k_input)

In [ ]:
pwxml_BAs = PWxml('pymatgen/io/espresso/tests/data/BAs/scf.xml')


In [ ]:
bohr_to_ang = 0.5291772105638411
pwxml_BAs = PWxml('pymatgen/io/espresso/tests/data/BAs/bands.xml')
alat_xml_input = 6.439670622385e0
alat_xml_output = pwxml_BAs.alat

import xmltodict
with open('pymatgen/io/espresso/tests/data/BAs/bands.xml') as fd:
    doc = xmltodict.parse(fd.read())['qes:espresso']


#alat = alat_xml_output
#T = pwxml_BAs.final_structure.lattice.reciprocal_lattice.matrix
#T = np.linalg.inv(T).T
#_, k_cart, _ = PWxml._parse_kpoints(doc['output'], T, alat) 
#k_output_cart = k_cart * bohr_to_ang
k_output_cart = pwxml_BAs.kpoints_cart

alat = alat_xml_input
kpts = doc['input']['k_points_IBZ']['k_point']
nk = len(kpts)
k = np.zeros((nk, 3), float)
k_weights = np.zeros(nk, float)
for n, kp in enumerate(kpts):
    k[n] = parse_pwvals(kp["#text"])
k_input_cart = k * (2 * np.pi / alat) * (1 / bohr_to_ang)

for k in zip(k_output_cart, k_input_cart):
    print(f'O: {np.around(k[0], 5)} ||| I: {np.around(k[1], 5)}')

In [ ]:
i = 60
k_output_cart[i]/k_input_cart[i]

In [ ]:
og.to(filename='BaAs_ella_og.cif')
mod.to(filename='BaAs_ella_mod.cif')

In [ ]:
pwin_Si = PWin.from_file('pymatgen/io/espresso/tests/data/Si/bands.in')
pwin_Ni = PWin.from_file('pymatgen/io/espresso/tests/data/Ni/bands.in')
pwin_0 = PWin.from_file('pymatgen/io/espresso/tests/data/Sr3PbO/bands.in')

In [ ]:
pwin_0.system
pwin_0.cell_parameters['options']
#pwin_0.atomic_positions['options']
#[x['symbol'] for x in pwin_0.atomic_positions['data']]
#[x['position'] for x in pwin_0.atomic_positions['data']]

In [ ]:
pwin_Si = PWin.from_file('pymatgen/io/espresso/tests/data/Si/bands.in')
structure = pwin_Si.to_structure()
structure.to(filename='Si.cif')
structure = pwin_Ni.to_structure()
structure.to(filename='Ni.cif')
structure = pwin_0.to_structure()
structure.to(filename='Sr3PbO.cif')
structure

In [ ]:
from pymatgen.electronic_structure.bandstructure import Kpoint
from pymatgen.io.espresso.pwxml import PWxml

path = 'pymatgen/io/espresso/tests/data/Bi2Te3/scf.xml'
pwxml = PWxml(path)

# This should work exactly as is with VASP btw
#reciprocal_lattice = pwxml.initial_structure.lattice.reciprocal_lattice
#S = np.linalg.inv(reciprocal_lattice.matrix).T # Matrix to convert from cartesian to fractional
#k_cart = np.array(pwxml.kpoints_cart)
#k_frac = (S @ k_cart.T).T
#k_frac_og = pwxml.kpoints_frac
#for i, k in enumerate(k_frac):
#    print(f"{k_frac_og[i]} vs {k}")

#for k in pwxml.kpoints_frac:
#    shifts = [0.01, 0.03, 0.04] # dx, dy, dz in cartesian (inverse angstrom)
#    shifts = T @ shifts # Convert to fractional
#    k_shifted = [k + dk for dk in shifts] # 3 shifted kpoints
#    for k_s in k_shifted:
#        # Check if all components of k_s are less than 1 and greater than 0
#        if all([0 <= k < 1 for k in k_s]):
#            print(f"{k_s} in BZ.")
#        else:
#            k_in_ibz = Kpoint(k, reciprocal_lattice, to_unit_cell=True)
#            print(f"{k_s} outside of BZ., in IBZ: {k_in_ibz.frac_coords}")



for i in range(11):
    for j in range(11):
        for k in range(5):
            datastruct[i, j, k] = full_kpts
#for i in range(grid[0]):
#    for j in range(grid[1]):
#        for k in range(grid[2]):
#            datastruct[i, j, k] = full_kpts
#

In [ ]:
from pymatgen.io.espresso.pwxml import PWxml
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.bandstructure import Kpoint

pwxml = PWxml("pymatgen/io/espresso/tests/data/Bi2Te3/scf.xml")
struct = pwxml.initial_structure
sg = SpacegroupAnalyzer(struct)
grid = (99, 99, 21) # Can read from XML, will implement
shift = (0, 0, 0) # Can read from XML, will implement
# These are the k-points in the IBZ
ibz_kpts = sg.get_ir_reciprocal_mesh(grid, is_shift=shift)
ibz_kpts = [ibz_kpts[i][0] for i in range(len(ibz_kpts))] # Get rid of weights 

# These are the k-points in the entire BZ
full_kpts, map = sg.get_ir_reciprocal_mesh_map(grid, is_shift=shift)
kpts = full_kpts.reshape(grid + (3,), order="F")
map = map.reshape(grid, order="F") # Maps kpts to IBZ

# Put these into shape Nx * Ny * Nz * 3 with proper ordering
for i in range(3):
    kpts = np.roll(kpts, int(np.floor(grid[i] / 2)), axis=i)
    map = np.roll(map, int(np.floor(grid[i] / 2)), axis=i)

ibz_equiv = full_kpts[map] # equivalent k-point in IBZ


# Create data structure
datastruct = [[[{}] * grid[2]] * grid[1]] * grid[0]
for i in range(grid[0]):
    datastruct.append([])
    for j in range(grid[1]):
        for k in range(grid[2]):
            kpt = kpts[i, j, k] 
            kpt_ibz = ibz_equiv[i, j, k] # equivalent k-point in IBZ
            # figure out index in ibz from map and ibz_kpts
            # QE likely uses same indexing as pymatgen, but check
            index = 1
            k_dict = {
                "kpt": Kpoint(kpt, pwxml.initial_structure.lattice.reciprocal_lattice),
                "kpt_ibz": Kpoint(kpt_ibz, pwxml.initial_structure.lattice.reciprocal_lattice),
                "wfc_file": f"wfc{index}.hdf5",
            }
            datastruct[i][j][k] = k_dict

In [ ]:
kpt_ibz = ibz_equiv[0, 0, 0] # equivalent k-point in IBZ
final_map = np.argwhere(np.isin(ibz_kpts, kpt_ibz).all(axis=1))
final_map

In [ ]:
print(kpt_ibz)
print(ibz_kpts[164])

In [ ]:
ibz_equiv.shape

In [ ]:
np.isin(ibz_kpts, ibz_equiv)

In [ ]:
# Find the index in ibz_kpts that corresponds to the k-point in the full BZ
# This is the index of the k-point in the IBZ
# Write function to find index in 2D array where row is equal to a given row
#def find_index(array, row):
#    for i, r in enumerate(array):
#        if np.all(r == row):
#            return i
#    return None


In [ ]:
nx = grid[0]
ny = grid[1]
nz = grid[2]
gamma = kpts[nx//2, ny//2, nz//2]
gamma

In [ ]:
path = 'pymatgen/io/espresso/tests/data/Ni/'
projwfc_xml = Projwfc.from_xml(path+'atomic_proj.xml', parse_projections=True)
projwfc_out = Projwfc.from_projwfcout(path+'projwfc.out', parse_projections=True)
projwfc_filproj = Projwfc.from_filproj(path+'filproj.projwfc_up', parse_projections=True)
#projwfc.atomic_states[Spin.up][0].projections
# Merge kpoints and weights into a single array
#projwfc.atomic_states
#states = projwfc
#states
#projwfc_xml.atomic_states[Spin.up][1].projections[0]
#phi_psi = np.array(states[0][0]['atomic_states']['phi_psi'])
#phi_psi = phi_psi[::2] + 1j*phi_psi[1::2]
#np.abs(phi_psi)**2
from_file = """
 -0.988044064409455       1.131845521650903E-003
  1.262043247685085E-011 -4.622602647885721E-012
  1.452471476426354E-011  7.515363331656033E-012
  2.509740679168537E-012 -6.942578456570203E-012
  0.154076525181495       5.133949657975312E-003
 -1.038064079139644E-011  3.109512647370138E-012
 -4.778204307914757E-011  2.881745536642555E-011
  1.678866073939744E-011  5.931233282296944E-011
  6.654751402712655E-012 -7.877149453550114E-012
 -2.197089732369761E-012 -3.503376061475549E-011
"""
print(projwfc_out.atomic_states[Spin.up][0].projections[0])
print(np.around(projwfc_filproj.atomic_states[Spin.up][0].projections[0], decimals=3))
print(np.around(projwfc_xml.atomic_states[Spin.up][0].projections[0], decimals=3))

In [ ]:
path = 'pymatgen/io/espresso/tests/data/Ni/'
projwfc_xml = Projwfc.from_xml(path+'atomic_proj.xml', parse_projections=True)
projwfc_out = Projwfc.from_projwfcout(path+'projwfc.out', parse_projections=True)
projwfc_filproj = Projwfc.from_filproj(path+'filproj.projwfc_up', parse_projections=True)
#projwfc.atomic_states[Spin.up][0].projections
# Merge kpoints and weights into a single array
#projwfc.atomic_states
#states = projwfc
#states
states = projwfc_xml
#projwfc_xml.atomic_states[Spin.up][1].projections[0]
phi_psi = np.array(states[0][0][0]['phi_psi'])
phi_psi = phi_psi[::2] + 1j*phi_psi[1::2]
np.abs(phi_psi)**2

In [ ]:
from io import StringIO
natomwfc = 8
states = []
selections = np.arange(1, 3)
filename = 'pymatgen/io/espresso/tests/data/Si/atomic_proj.xml'
for event, elem in ET.iterparse(filename, events=("start", "end")):
    if elem.tag == "HEADER" and event == "end":
        print("HEADERRRRRR")
    if event == 'end' and elem.tag == 'PROJECTIONS':
        for e in elem.iter('ATOMIC_WFC'):
            state_i = int(e.attrib["index"])
            if int(e.attrib['index']) == 1:
                # New kpoint
                kpt = [None]*natomwfc
            elif int(e.attrib['index']) == natomwfc:
                # End of kpoint
                states.append(kpt)
            if state_i in selections:
                s = {
                        "state_i": state_i,
                        "spin": int(e.attrib["spin"]),
                        "phi_psi": np.array(parse_pwvals(e.text)),
                    }
                kpt[state_i - 1] = s


In [ ]:
#import xmltodict
#with open('pymatgen/io/espresso/tests/data/Sr3PbO/atomic_proj.xml') as fd:
#    doc = xmltodict.parse(fd.read())
#k_i = 0
#state_i = 0
#projections = parse_pwvals(doc['PROJECTIONS']['EIGENSTATES']['PROJS'][k_i]['ATOMIC_WFC'][state_i]['#text'])
#projections
## Form complesx number array from projections. Element 0 is real, 1 is imaginary, 2 is real, etc.
#projections = np.array([complex(projections[i], projections[i+1]) for i in range(0, len(projections), 2)])
#projections.shape

In [ ]:
import numpy as np
x = np.arange(1, 11)
x[::2] + x[1::2]*1j

 # Sr3PbO
 noncolinear+SOC band structure

In [ ]:
dir = 'Sr3PbO'
xml_filename = f'pymatgen/io/espresso/tests/data/{dir}/bands.xml'
in_filename = f'pymatgen/io/espresso/tests/data/{dir}/bands.in'
pwxml = PWxml(xml_filename)
pwin = PWin.from_file(in_filename)
#print(pwin.to_str())

bs = pwxml.get_band_structure()
BSPlotter(bs).show()


In [ ]:
debug = pwxml._debug


 # Si
 colinear band structure, but ibrav != 0 with kpoints in crystal_b
 This causes the same issue as the Ni calculation with pwxml.actual_kpoints being different from those in the actual input file


 scf.xml has as many bands as electrons/2, which should trigger a warning since such cases break BSPlotter()

In [ ]:
dir = 'Si'
xml_filename = f'pymatgen/io/espresso/tests/data/{dir}/bands.xml'
in_filename = f'pymatgen/io/espresso/tests/data/{dir}/bands.in'
pwxml = PWxml(xml_filename)
pwin = PWin.from_file(in_filename)
#print(pwin.to_str())

bs = pwxml.get_band_structure(kpoints_filename=in_filename)
BSPlotter(bs).show()

#pwxml.actual_kpoints


 # Al
 collinear bands with ibrav != 0 and tpiba cartesian k points with no option for the K_POINTS card (i.e., need to parse default)

 the ibrav = 2 seems to cause issues with the actual kpoints computed are different from those in the K_POINTS card (symm. equiv)

In [ ]:
dir = 'Al'
xml_filename = f'pymatgen/io/espresso/tests/data/{dir}/bands.xml'
in_filename = f'pymatgen/io/espresso/tests/data/{dir}/bands.in'
pwxml = PWxml(xml_filename)
pwin = PWin.from_file(in_filename)
#print(pwin.to_str())

bs = pwxml.get_band_structure(kpoints_filename=in_filename)
BSPlotter(bs).show()


 # Ni
 collinear, spin-polarized bands, with ibrav = 2

 the ibrav = 2 seems to cause issues with the actual kpoints computed are different from those in the K_POINTS card (symm. equiv)

In [ ]:
dir = 'Ni'
xml_filename = f'pymatgen/io/espresso/tests/data/{dir}/bands.xml'
in_filename = f'pymatgen/io/espresso/tests/data/{dir}/bands.in'
pwxml = PWxml(xml_filename)
pwin = PWin.from_file(in_filename)
#print(pwin.to_str())

bs = pwxml.get_band_structure()
BSPlotter(bs).show()


 # FeO
 collinear, spin-polarized scf, with DFT+U

In [ ]:
dir = 'FeO'
xml_filename = f'pymatgen/io/espresso/tests/data/{dir}/scf.xml'
pwxml = PWxml(xml_filename)
#print(pwin.to_str())


 # MoS2
 collinear scf, with vdW

In [ ]:
dir = 'MoS2'
xml_filename = f'pymatgen/io/espresso/tests/data/{dir}/scf.xml'
pwxml = PWxml(xml_filename)
#print(pwin.to_str())


 # Debugging

In [ ]:
dir = 'Si'
calc = 'bands'
xml_filename = f'pymatgen/io/espresso/tests/data/{dir}/{calc}.xml'
with open(xml_filename, encoding="UTF-8") as xml:
    data = xmltodict.parse(xml.read())['qes:espresso']
#data['output']['band_structure']
data['output']['atomic_structure']


In [ ]:
vasprun_filename = f'pymatgen/io/espresso/tests/data/Si/vasprun_bands.xml'
vasprun_frac = Vasprun(vasprun_filename)#, parse_projected_eigen=True)
kpoints_filename = f'pymatgen/io/espresso/tests/data/Si/KPOINTS_bands'
#KP = Kpoints.from_file(kpoints_filename)
#bs = vasprun.get_band_structure(kpoints_filename=kpoints_filename)
#from pymatgen.electronic_structure.plotter import BSPlotter
#BSPlotter(bs).show()
bs_frac = vasprun_frac.get_band_structure(kpoints_filename=kpoints_filename)
alat = vasprun_frac.final_structure.lattice.a
[k.cart_coords*alat/2/np.pi for k in bs.kpoints[::16]]
#vasprun.actual_kpoints


In [ ]:
#projwfc = Projwfc.from_filproj('pymatgen/io/espresso/tests/data/Sr3PbO/bands.proj')
#projwfc = Projwfc.from_filproj('pymatgen/io/espresso/tests/data/Si/Si.projwfc_up')
#projwfc = Projwfc.from_filproj('pymatgen/io/espresso/tests/data/Bi2Te3/Bi2Te3.projwfc_up')
projwfc = Projwfc.from_filproj('pymatgen/io/espresso/tests/data/Ni/filproj.projwfc_down')
#pwxml = PWxml('pymatgen/io/espresso/tests/data/Si/scf.xml')
#pwxml.final_structure



In [ ]:
projwfc


In [ ]:
pwxml = PWxml('pymatgen/io/espresso/tests/data/Ni/bands.xml')


In [ ]:
pwxml.eigenvalues[Spin.up].shape


In [ ]:
data = Projwfc.from_projwfcout('pymatgen/io/espresso/tests/data/Ni/projwfc.out')
data.atomic_states

In [ ]:
projwfc = projwfc.from_filproj('pymatgen/io/espresso/tests/data/Sr3PbO/filproj.projwfc_up')
#projwfc.atomic_states

In [ ]:
#np.isclose(projections, projections_filproj, atol=1e-3).all()


In [ ]:
nkstot = 10
12 // nkstot


In [ ]:
text = """ArithmeticError
AssertionError
AttributeError"""

import re
found = re.findall(r'(\w+Error)', text)
if found:
    print("found")




In [ ]:
from pymatgen.electronic_structure.core import Magmom, Orbital, OrbitalType, Spin


In [ ]:
Spin.up
